Many examples are extensions of https://github.com/PacktPublishing/Interactive-Dashboards-and-Data-Apps-with-Plotly-and-Dash
which are from [Interactive Dashboards and Data Apps with Plotly and Dash](https://www.packtpub.com/product/interactive-dashboards-and-data-apps-with-plotly-and-dash/9781800568914?utm_source=github&utm_medium=repository&utm_campaign=9781800568914)

In [92]:
import intersystems_iris.dbapi._DBAPI as dbapi
from dotenv import load_dotenv
load_dotenv()
import os

# See example from https://github.com/intersystems-community/intersystems-irispython
# Is some issues with the other driver, so use this one.

password = os.getenv("SQL_PASSWORD")
user = os.getenv("SQL_USERNAME")
host = os.getenv("SQL_SERVER")
namespace = os.getenv("SQL_NAMESPACE")
port = os.getenv("SQL_PORT")
if isinstance(port, str):
    port = int(port)

config = {
    "hostname": host,
    "port": port,
    "namespace": namespace,
    "username": user,
    "password": password,
}

try:
    conn = dbapi.connect(**config)
    print("Connection successful!")
except Exception as e:
    print(f"Failed to connect: {e}")

# create a cursor
cursor = conn.cursor()
cursor


Connection successful!


In [88]:
import pandas as pd

sql = """
      select ResourceType, count(*) from HSFHIR_X0001_R.Rsrc group by ResourceType
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
df

,ResourceType,Aggregate_2
0,Binary,100
1,DiagnosticReport,8
2,DocumentReference,8
3,Encounter,12
4,Observation,2
5,Organization,15
6,Patient,10
7,ServiceRequest,11
8,Specimen,3


In [89]:
sql = """
      select dr.ID1,CDR.GetJSON(ResourceString,'basedOn'),$PIECE(CDR.GetFHIRPathOne(ResourceString,'basedOn[0].reference','DiagnosticReport'),'/',2) ServiceRequest,sr.requester_IdentifierValue from HSFHIR_X0001_S.DiagnosticReport dr
  join HSFHIR_X0001_R.Rsrc resource on resource.ResourceType = 'DiagnosticReport'
          and resource.Key = dr.Key
  left outer join HSFHIR_X0001_S.ServiceRequest sr on CDR.GetFHIRPathOne(ResourceString,'basedOn[0].reference','DiagnosticReport')=sr.key
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)

df1 = df.groupby(['requester_IdentifierValue']).size().reset_index(name='count')

df1

,requester_IdentifierValue,count
0,R0A,2
1,REP,1
2,RWJ,1
3,RX1,1


In [90]:
sql = """
      select
          dr.ID1
           ,sr.requester_IdentifierValue ODS
           ,org.phonetic name
           ,org.partof_IdentifierValue ICS
           ,parent.phonetic ICSName
      from HSFHIR_X0001_S.DiagnosticReport dr                                             join HSFHIR_X0001_S_DiagnosticReport.basedOn drb on drb.Key = dr.Key               left outer join HSFHIR_X0001_S.ServiceRequest sr on drb.value_Reference=sr.key
      left outer join HSFHIR_X0001_S.Organization org on org.key = sr.requester_RelativeRef                                                             left outer join HSFHIR_X0001_S.Organization parent on parent.key = org.partof_Reference
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)

df1 = df.groupby(['ODS','name','ICS','ICSName']).size().reset_index(name='count')

df1

,ODS,name,ICS,ICSName,count
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,QOP,NHS GREATER MANCHESTER INTEGRATED CARE BOARD,2
1,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST,QYG,NHS CHESHIRE AND MERSEYSIDE INTEGRATED CARE BOARD,1
2,RWJ,STOCKPORT NHS FOUNDATION TRUST,QOP,NHS GREATER MANCHESTER INTEGRATED CARE BOARD,1
3,RX1,NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST,QT1,NHS NOTTINGHAM AND NOTTINGHAMSHIRE INTEGRATED ...,1


In [91]:
sql = """
      select
          dr.ID1
           ,pat.value_Value MRN
           ,pat2.value_Value NHS
           ,sri.value_Value OrderPlacer
           ,sr.requester_IdentifierValue ODS
           ,org.phonetic name
           ,org.partof_IdentifierValue ICS
           ,parent.phonetic ICSName
      from HSFHIR_X0001_S.DiagnosticReport dr                                                                                                 join HSFHIR_X0001_S_DiagnosticReport.basedOn drb on drb.Key = dr.Key                                                                                   left outer join HSFHIR_X0001_S.ServiceRequest sr on drb.value_Reference=sr.key
                                                                                                                                              left outer join HSFHIR_X0001_S_ServiceRequest.identifier
          sri on sr.key=sri.key and sri.value_TypeCodingCode='PLAC'
                                                                                                                                              left outer join HSFHIR_X0001_S.Organization org on org.key = sr.requester_RelativeRef                                                                                           left outer join HSFHIR_X0001_S.Organization parent on parent.key = org.partof_Reference
                                                                                                                                              left outer join HSFHIR_X0001_S_Patient.identifier pat on pat.key=dr.patient_Reference and pat.value_TypeCodingCode = 'MR'
                                                                                                                                              left outer join HSFHIR_X0001_S_Patient.identifier pat2 on pat2.key=dr.patient_Reference and pat2.value_TypeCodingCode = 'NH'
      where org.partof_IdentifierValue IS NOT NULL
      order by sr.requester_IdentifierValue
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)

df

,ID1,MRN,NHS,OrderPlacer,ODS,name,ICS,ICSName
0,4,None,None,1028515276,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,QOP,NHS GREATER MANCHESTER INTEGRATED CARE BOARD
1,5,None,None,1029467053,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,QOP,NHS GREATER MANCHESTER INTEGRATED CARE BOARD
2,7,20002983,9999999522,None,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST,QYG,NHS CHESHIRE AND MERSEYSIDE INTEGRATED CARE BOARD
3,6,None,None,41070,RWJ,STOCKPORT NHS FOUNDATION TRUST,QOP,NHS GREATER MANCHESTER INTEGRATED CARE BOARD
4,2,None,9737023854,1000152868,RX1,NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST,QT1,NHS NOTTINGHAM AND NOTTINGHAMSHIRE INTEGRATED ...
5,2,None,9737023854,1000152868,RX1,NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST,QT1,NHS NOTTINGHAM AND NOTTINGHAMSHIRE INTEGRATED ...
6,2,None,9737023854,1000152868,RX1,NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST,QT1,NHS NOTTINGHAM AND NOTTINGHAMSHIRE INTEGRATED ...
7,2,None,9737023854,1000152868,RX1,NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST,QT1,NHS NOTTINGHAM AND NOTTINGHAMSHIRE INTEGRATED ...
8,2,None,9737023854,1000152868,RX1,NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST,QT1,NHS NOTTINGHAM AND NOTTINGHAMSHIRE INTEGRATED ...
9,2,None,9737023854,1000152868,RX1,NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST,QT1,NHS NOTTINGHAM AND NOTTINGHAMSHIRE INTEGRATED ...
